In [6]:
import pandas as pd
import glob
from google.cloud import translate_v2 as translate
import os
from openai import OpenAI


In [11]:
# Get list of all xlsx files in current directory
xlsx_files = glob.glob('*.xlsx')
print("Found xlsx files:", xlsx_files)

# Read each xlsx file into a dictionary of dataframes and clean line breaks
dfs = {}
for file in xlsx_files:
    print(f"Attempting to read file: {file}")
    # Get country name from filename by removing "Holocaust Encyclopedia - search queries from " and " 2024"
    country = file.replace('Holocaust Encyclopedia - search queries from ', '').replace(' 2024.xlsx', '')
    
    # Read all sheets from Excel file
    excel_file = pd.ExcelFile(file)
    
    # Process each sheet
    for sheet in excel_file.sheet_names:
        # Read sheet
        df = pd.read_excel(file, sheet_name=sheet)
        
        # Clean line breaks
        df = df.replace('\n', ' ', regex=True)
        
        # Create filename: country_sheetname_2024.csv (all lowercase)
        filename = f"{country.lower()}_{sheet.lower()}_2024.csv"
        
        # Save to CSV
        df.to_csv(filename, index=False)
        
        # Store in dictionary
        dfs[filename.replace('.csv','')] = df

Found xlsx files: ['Holocaust Encyclopedia - search queries from Turkey 2024.xlsx', 'Holocaust Encyclopedia - search queries from Russia 2024.xlsx', 'Holocaust Encyclopedia - search queries from China 2024.xlsx', 'Holocaust Encyclopedia - search queries from United States 2024.xlsx', 'Holocaust Encyclopedia - search queries from Germany 2024.xlsx', 'Holocaust Encyclopedia - search queries from South Korea 2024.xlsx']
Attempting to read file: Holocaust Encyclopedia - search queries from Turkey 2024.xlsx
Attempting to read file: Holocaust Encyclopedia - search queries from Russia 2024.xlsx
Attempting to read file: Holocaust Encyclopedia - search queries from China 2024.xlsx
Attempting to read file: Holocaust Encyclopedia - search queries from United States 2024.xlsx
Attempting to read file: Holocaust Encyclopedia - search queries from Germany 2024.xlsx
Attempting to read file: Holocaust Encyclopedia - search queries from South Korea 2024.xlsx


In [12]:
# Get list of all query CSV files
query_files = glob.glob('*_queries_2024.csv')

# Initialize empty list to store dataframes
query_dfs = []

# Read each query file and add country column
for file in query_files:
    # Extract country name and year from filename
    country = file.split('_')[0]
    year = file.split('_')[-1].replace('.csv', '')
    
    # Read CSV
    df = pd.read_csv(file)
    
    # Add country and year columns
    df['Country'] = country
    df['Year'] = year
    
    # Append to list
    query_dfs.append(df)

# Concatenate all dataframes
all_queries = pd.concat(query_dfs, ignore_index=True)

# Display first few rows
print("Shape of combined dataframe:", all_queries.shape)
all_queries.head()



Shape of combined dataframe: (6000, 7)


,Top queries,Clicks,Impressions,CTR,Position,Country,Year
0,yunanistan haritası,3666,82458,0.0445,1.56,turkey,2024
1,ermeni soykırımı,3601,40171,0.0896,3.05,turkey,2024
2,itilaf devletleri,3282,236362,0.0139,1.07,turkey,2024
3,adolf hitler,3168,605973,0.0052,3.87,turkey,2024
4,ermeni soykırımı nedir,2893,21421,0.1351,2.80,turkey,2024


In [12]:
pip install deep_translator

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 42 kB 1.9 MB/s eta 0:00:011
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Test with just first 5 rows
test_df = all_queries.head(5).copy()

from deep_translator import GoogleTranslator

def translate_text(text):
    """Translates text to English"""
    if pd.isna(text):
        return text
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        print(f"Error translating {text}: {e}")
        return text

# Create English query column for test data
test_df['English Google Translate'] = test_df['Top queries'].apply(translate_text)

# Display original and translated queries
print("\nSample of translated queries:")
print(test_df[['Top queries', 'English Google Translate']])


Sample of translated queries:
              Top queries                  Query_English
0     yunanistan haritası                     Greece map
1        ermeni soykırımı              Armenian Genocide
2       itilaf devletleri                 Entente States
3            adolf hitler                   adolf hitler
4  ermeni soykırımı nedir  What is the Armenian genocide


In [ ]:
from deep_translator import GoogleTranslator
from tqdm import tqdm
import csv
import os

def translate_text(text):
    """Translates text to English"""
    if pd.isna(text):
        return text
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        print(f"Error translating {text}: {e}")
        return text

# Create output file with headers
output_file = 'translated_queries.csv'
headers = ['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position', 'Country', 'Year', 'English Google Translate']
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(headers)

# Process in batches and append to CSV
batch_size = 100
print("\nTranslating queries...")
for i in tqdm(range(0, len(all_queries), batch_size)):
    batch = all_queries.iloc[i:i+batch_size].copy()
    batch['English Google Translate'] = batch['Top queries'].apply(translate_text)
    
    # Append batch to CSV
    batch.to_csv(output_file, mode='a', header=False, index=False)

# Display sample of original and translated queries
print("\nSample of translated queries:")
sample_df = pd.read_csv(output_file, nrows=10)
print(sample_df[['Top queries', 'English Google Translate']])

all_queries = pd.read_csv(output_file)# Reorder columns to put translation next to original
column_order = ['Top queries','English Google Translate', 'Clicks', 'Impressions', 'CTR', 'Position', 'Country', 'Year'] 
all_queries = all_queries[column_order]
all_queries.to_csv(output_file, index=False)

In [28]:
all_queries = pd.read_csv(output_file)# Reorder columns to put translation next to original
column_order = ['Top queries','English Google Translate', 'Clicks', 'Impressions', 'CTR', 'Position', 'Country', 'Year'] 
all_queries = all_queries[column_order]
all_queries.to_csv(output_file, index=False)

In [50]:
# Get OpenAI API key from zshrc
openai_api_key = os.popen('source ~/.zshrc && echo $OPENAI_API_KEY').read().strip()

# Set as environment variable
os.environ['OPENAI_API_KEY'] = openai_api_key

# Initialize OpenAI client
client = OpenAI()

# Initialize cost tracking
total_tokens = 0
total_cost = 0
COST_PER_1K_TOKENS = 0.002  # Cost for gpt-4o-mini

def get_gpt_translation(text):
    """Translates text using GPT-3.5"""
    global total_tokens, total_cost
    
    if pd.isna(text):
        return text
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Most cost-effective model
            messages=[
                {"role": "system", "content": "You are a translator. Translate the following text to English. Do not add any additional text or comments. If it is already in English, just return the text."},
                {"role": "user", "content": text}
            ],
            temperature=0.3  # Lower temperature for more consistent translations
        )
        
        # Update token counts and cost
        tokens_used = response.usage.total_tokens
        total_tokens += tokens_used
        cost = (tokens_used / 1000) * COST_PER_1K_TOKENS
        total_cost += cost
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error translating {text}: {e}")
        return text

In [ ]:

# Create output file with headers
column_order = ['Top queries', 'English Google Translate', '4o-mini Translate', 'Clicks', 'Impressions', 'CTR', 'Position', 'Country', 'Year']
pd.DataFrame(columns=column_order).to_csv('translated_queries_gpt.csv', index=False)



In [54]:
# Process in batches of 5
print(f"\nTranslating all queries with GPT-4o-mini...")
batch_size = 5
for i in tqdm(range(280, len(all_queries), batch_size), desc="Translating"):
    # Get batch
    batch = all_queries.iloc[i:i+batch_size].copy()
    
    # Translate batch
    for j in range(len(batch)):
        text = batch.iloc[j]['Top queries']
        translation = get_gpt_translation(text)
        # Remove any newlines from the translation
        translation = translation.replace('\n', ' ').strip()
        batch.at[batch.index[j], '4o-mini Translate'] = translation
    # Reorder columns
    batch = batch[column_order]
    
    # Append batch to CSV
    batch.to_csv('translated_queries_gpt.csv', mode='a', header=False, index=False)
    
    # Print running cost
    print(f"\nTokens used so far: {total_tokens}")
    print(f"Total cost so far: ${total_cost:.4f}")

# Print final usage statistics
print(f"\nFinal Statistics:")
print(f"Total tokens used: {total_tokens}")
print(f"Total cost: ${total_cost:.4f}")

# Read final results to display sample
final_results = pd.read_csv('translated_queries_gpt.csv')
print("\nComparing Google and GPT translations:")
print(final_results[['Top queries', 'English Google Translate', '4o-mini Translate']].head(10))


Translating:   0%|          | 0/1144 [00:00<?, ?it/s]


Translating all queries with GPT-4o-mini...


Translating:   0%|          | 1/1144 [00:02<43:12,  2.27s/it]


Tokens used so far: 23249
Total cost so far: $0.0465


Translating:   0%|          | 2/1144 [00:04<40:57,  2.15s/it]

...


Tokens used so far: 379796
Total cost so far: $0.7596


Translating: 100%|█████████▉| 1143/1144 [56:28<00:04,  4.35s/it]


Tokens used so far: 380084
Total cost so far: $0.7602


Translating: 100%|██████████| 1144/1144 [56:31<00:00,  2.96s/it]


Tokens used so far: 380432
Total cost so far: $0.7609

Final Statistics:
Total tokens used: 380432
Total cost: $0.7609

Comparing Google and GPT translations:
              Top queries       English Google Translate  \
0     yunanistan haritası                     Greece map   
1        ermeni soykırımı              Armenian Genocide   
2       itilaf devletleri                 Entente States   
3            adolf hitler                   adolf hitler   
4  ermeni soykırımı nedir  What is the Armenian genocide   
5        osmanlı haritası                    Ottoman map   
6         2. dünya savaşı                  2nd World War   
7         1. dünya savaşı                   World War II   
8   1. dünya savaşı özeti          1st World War Summary   
9                  hitler                         hitler   

                4o-mini Translate  
0                   map of Greece  
1               Armenian genocide  
2                   Allied Powers  
3                    Adolf Hitler  

In [ ]:
# Read the CSV file
df = pd.read_csv('translated_queries_gpt.csv')

# Calculate file_row_index (1-based indexing within each file of 1000 rows)
df['file_row_index'] = (df.index % 1000) 

# Save back to CSV
df.to_csv('translated_queries_gpt.csv', index=False)

# Display first few rows to verify
print("\nAdded file_row_index column:")
print(df[['Top queries', 'file_row_index']].head())


Updated file_row_index to start at 0 for each 1000 rows:
              Top queries  file_row_index
0     yunanistan haritası               0
1        ermeni soykırımı               1
2       itilaf devletleri               2
3            adolf hitler               3
4  ermeni soykırımı nedir               4
5        osmanlı haritası               5
6         2. dünya savaşı               6
7         1. dünya savaşı               7
8   1. dünya savaşı özeti               8
9                  hitler               9


In [61]:
# Read the CSV file again to ensure we have the latest data
df = pd.read_csv('translated_queries_gpt.csv')

# Define the desired country order
country_order = ['united states', 'germany', 'south korea', 'russia', 'turkey', 'china']

# Convert country names to lowercase for consistent matching
df['Country'] = df['Country'].str.lower()

# Create empty list to store country dataframes
country_dfs = []

# Split into separate dataframes by country while preserving chunk order
for country in country_order:
    country_df = df[df['Country'] == country].copy()
    country_df['file_row_index'] = country_df.index % 1000  # Maintain position within chunks
    country_dfs.append(country_df)

# Concatenate all country dataframes in desired order
df = pd.concat(country_dfs)

# Save back to CSV
df.to_csv('translated_queries_gpt2.csv', index=False)

# Display first few rows to verify
print("\nSorted by country while preserving chunk order:")
print(df[['Top queries', 'Country', 'file_row_index']].head())



Sorted by country while preserving chunk order:
                                Top queries        Country  file_row_index
5000  how many people died in the holocaust  united states               0
5001                      armenian genocide  united states               1
5002                 holocaust encyclopedia  united states               2
5003                    first they came for  united states               3
5004                              holocaust  united states               4


In [64]:
# Read the CSV file
all_queries = pd.read_csv('translated_queries_gpt2.csv')

# Add sequential ID column starting from 0
all_queries['id'] = range(len(all_queries))

# Display first few rows to verify
print("\nAdded sequential ID column:")
print(all_queries[['id', 'Top queries', 'Country', 'file_row_index']].head())

# Save back to CSV with all columns including the new ID column
all_queries.to_csv('translated_queries_gpt2.csv', index=False)



Added sequential ID column:
   id                            Top queries        Country  file_row_index
0   0  how many people died in the holocaust  united states               0
1   1                      armenian genocide  united states               1
2   2                 holocaust encyclopedia  united states               2
3   3                    first they came for  united states               3
4   4                              holocaust  united states               4


In [66]:
# Reorder columns to put id first
cols = ['id'] + [col for col in all_queries.columns if col != 'id']
all_queries = all_queries[cols]

# Display first few rows to verify
print("\nReordered columns with id first:")
print(all_queries.head())

# Save to file
all_queries.to_csv('translated_queries_gpt2.csv', index=False)



Reordered columns with id first:
   id                            Top queries  \
0   0  how many people died in the holocaust   
1   1                      armenian genocide   
2   2                 holocaust encyclopedia   
3   3                    first they came for   
4   4                              holocaust   

                English Google Translate  \
0  how many people died in the holocaust   
1                      armenian genocide   
2                 holocaust encyclopedia   
3                    first they came for   
4                              holocaust   

                                   4o-mini Translate  Clicks  Impressions  \
0  The Holocaust resulted in the deaths of approx...   40678       189593   
1                                  Armenian genocide   37535       447164   
2                             holocaust encyclopedia   35664        50277   
3                                first they came for   34702        77617   
4                          

In [29]:
all_queries.to_csv('translated_queries_gpt.csv', index=False)
